In [1]:
import os
import pandas as pd
import glob

from pathlib import Path

In [2]:
dir_path = Path(os.getcwd())
data_dir = str(os.path.join(dir_path.parent,'data'))
balance_files = glob.glob(data_dir + "/poker_balances*.csv")
hands_files = glob.glob(data_dir + "/poker_hands*.csv")

def get_hands_df():
    hands_dfs=[]

    for file in hands_files:
        new_df = pd.read_csv(file)
        hands_dfs.append(new_df)

    hands = pd.concat(hands_dfs)
    return hands
    
def get_balance_df():
    balance_dfs=[]
    
    for file in balance_files:
        new_df = pd.read_csv(file)
        balance_dfs.append(new_df)
        
    balances = pd.concat(balance_dfs)
    return balances


In [6]:
pd.set_option('display.max_columns', None)  
hands_base = get_hands_df()
games = get_balance_df()
hands = pd.merge(hands_base,games)

In [7]:
hands

,game_id,player_name,player_type,bet_number,opponents,call,current,final,pot,allowed,hand1,hand2,community1,community2,community3,community4,community5,game_result,game_reason,blind_type,beginning_balance,game_start_balance,game_end_balance,game_net_change
0,1,players_3,AlwaysCallPlayer,1,5,10,0,10.0,35,True,6-clubs,J-hearts,Z-N/A,Z-N/A,Z-N/A,Z-N/A,Z-N/A,won,won_game,None,100000,100000,100002,2
1,1,players_3,AlwaysCallPlayer,2,5,0,10,10.0,60,True,6-clubs,J-hearts,10-diamonds,4-clubs,K-hearts,Z-N/A,Z-N/A,won,won_game,None,100000,100000,100002,2
2,1,players_3,AlwaysCallPlayer,3,5,0,10,10.0,60,True,6-clubs,J-hearts,10-diamonds,4-clubs,4-spades,K-hearts,Z-N/A,won,won_game,None,100000,100000,100002,2
3,1,players_3,AlwaysCallPlayer,4,4,0,10,10.0,60,True,6-clubs,J-hearts,10-diamonds,4-clubs,4-spades,5-spades,K-hearts,won,won_game,None,100000,100000,100002,2
4,2,players_3,AlwaysCallPlayer,1,5,10,0,10.0,45,True,10-diamonds,Q-hearts,Z-N/A,Z-N/A,Z-N/A,Z-N/A,Z-N/A,won,won_game,None,100000,100002,100002,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014774,9000,players_2,AlwaysCallPlayer,5,5,10,30,40.0,200,True,8-clubs,A-clubs,3-spades,K-hearts,Q-spades,Z-N/A,Z-N/A,won,won_game,small,100000,100492,100492,0
1014775,9000,players_2,AlwaysCallPlayer,6,5,0,40,40.0,240,True,8-clubs,A-clubs,3-spades,K-diamonds,K-hearts,Q-spades,Z-N/A,won,won_game,small,100000,100492,100492,0
1014776,9000,players_2,AlwaysCallPlayer,7,5,0,40,40.0,240,True,8-clubs,A-clubs,2-clubs,3-spades,K-diamonds,K-hearts,Q-spades,won,won_game,small,100000,100492,100492,0
1014777,9000,players_2,AlwaysCallPlayer,8,5,10,40,50.0,260,True,8-clubs,A-clubs,2-clubs,3-spades,K-diamonds,K-hearts,Q-spades,won,won_game,small,100000,100492,100492,0


In [8]:
games

,game_id,player_name,player_type,game_result,game_reason,blind_type,beginning_balance,game_start_balance,game_end_balance,game_net_change
0,1,players_3,AlwaysCallPlayer,won,won_game,None,100000,100000,100002,2
1,2,players_3,AlwaysCallPlayer,won,won_game,None,100000,100002,100002,0
2,3,players_3,AlwaysCallPlayer,won,won_game,small,100000,100002,100002,0
3,4,players_3,AlwaysCallPlayer,won,won_game,big,100000,100002,100002,0
4,5,players_3,AlwaysCallPlayer,won,won_game,None,100000,100002,100002,0
...,...,...,...,...,...,...,...,...,...,...
5995,8996,players_2,AlwaysCallPlayer,won,won_game,None,100000,100492,100492,0
5996,8997,players_2,AlwaysCallPlayer,won,won_game,None,100000,100492,100492,0
5997,8998,players_2,AlwaysCallPlayer,won,won_game,None,100000,100492,100492,0
5998,8999,players_2,AlwaysCallPlayer,won,won_game,None,100000,100492,100492,0


# Basic Game Statistics

Number of Games played in the simulation

In [12]:
number_of_games = len(set(games.game_id))
number_of_games

25000

Players Mean number of chips gained and variance

In [13]:
chips_gained_per_game = games.groupby('player_name').mean('game_net_change')[['game_net_change']].reset_index()
variance_calculation = games[['player_name','game_net_change']]
variance_calculation['game_net_change'] = variance_calculation['game_net_change'] ** 2
expectation_squared = variance_calculation.groupby('player_name').mean('game_net_change').reset_index()
merged_expectations = pd.merge(expectation_squared,chips_gained_per_game,on='player_name')
merged_expectations['variance'] = merged_expectations['game_net_change_x'] - merged_expectations['game_net_change_y']
summary_stats = merged_expectations[['player_name','game_net_change_y','variance']]
summary_stats.columns = ['player_name','mean','variance']
summary_stats['games'] = number_of_games
summary_stats

c:\users\silbe\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,player_name,mean,variance,games
0,players_1,0.49168,3.18024,25000
1,players_2,0.49168,3.18024,25000
2,players_3,0.49168,3.18024,25000
3,players_4,0.49168,3.18024,25000
4,players_5,0.49168,3.18024,25000
5,players_6,-2.45840,94.25640,25000


Top 10 Best Card Hands Dealt by Mean Number of Chips Gained 

In [14]:
two_card_hands = hands[['game_id','player_name','player_type','hand1','hand2','game_net_change']]
two_card_hands = two_card_hands.drop_duplicates()
best_two_card_hands = two_card_hands.groupby(['player_name','hand1','hand2']).mean().reset_index()[['player_name','hand1','hand2','game_net_change']]
best_two_card_hands = best_two_card_hands.sort_values(by=['player_name','game_net_change'], ascending=[True,False])
best_two_card_hands.groupby('player_name').head(10)

,player_name,hand1,hand2,game_net_change
916,players_1,6-hearts,Q-clubs,2.615385
1158,players_1,9-diamonds,A-diamonds,2.588235
1079,players_1,8-diamonds,A-clubs,2.555556
1092,players_1,8-diamonds,Q-diamonds,2.181818
33,players_1,10-clubs,9-hearts,2.133333
523,players_1,3-spades,8-diamonds,2.000000
599,players_1,4-diamonds,8-clubs,2.000000
737,players_1,5-diamonds,7-clubs,2.000000
922,players_1,6-spades,7-hearts,2.000000
1144,players_1,9-clubs,J-diamonds,2.000000
